# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

In [695]:
import urllib2
import requests
import bs4
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

In [589]:
#def extract_text(el):
#    if el:
#        return el.text.strip()
#    else:
#        return ''
        

#def get_company_from_result(result):
#    return extract_text(result.find('span', {'class' : 'company'}))

#def get_location_from_result(result):
#    return  extract_text(result.find('span', {'class' : 'location'}))

#def get_summary_from_result(result):
#    return  extract_text(result.find('span', {'class' : 'summary'}))

#def get_title_from_result(result):
#    return result.find('a', {'data-tn-element' : 'jobTitle'}).text.strip()

#def get_salary_from_result(result):
#    salary_table = result.find('td', {'class' : 'snip'})
#    if salary_table:
#        snip = salary_table.find('nobr')
#        if snip:
#            return snip.text.strip()
        
#    return None

In [590]:
#url = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
#rows = []
#max_results_per_city = 5000

#for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Atlanta', 'Washington+DC', 
#                 'Minneapolis', 'Denver', 'Seattle', 'Boston', 'Los+Angeles']):
#    for start in range(0, max_results_per_city, 10):
#        r = requests.get(url.format(city, start))
#        soup = BeautifulSoup(r.content)
#        results = soup.findAll('div', { "class" : "result" })
#        for result in results:
#            if result:
#                row = {}
#                row['title'] = get_title_from_result(result)
#                row['company'] = get_company_from_result(result)
#                row['summary'] = get_summary_from_result(result)
#                row['salary'] = get_salary_from_result(result)
#                row['city'] = city
#                rows.append(row)
#
#import pandas as pd
#data = pd.DataFrame.from_records(rows)

#salary_data = data[data.salary.notnull()]
#salary_data = salary_data[~(salary_data.salary.astype('str').str.contains('hour'))]
#salary_data = salary_data[~(salary_data.salary.astype('str').str.contains('day'))]
#salary_data = salary_data.drop_duplicates()

In [612]:
#salary_data.salary.apply(str)

In [592]:
#salary_data = salary_data[~(salary_data.salary.astype('str').str.contains('month'))]
#salary_data.salary = salary_data['salary'].apply(lambda x: x.lstrip('$').rstrip('abcdefghijklmnopqrstuvwxyz'))        

In [593]:
#salary_data.to_csv('../assets/04-project4-assets/dataScienceSalaries.csv', encoding='utf8', index=False)

In [662]:
dsSalaries = pd.read_csv('../assets/04-project4-assets/dataScienceSalaries.csv')

In [663]:
dsSalaries2= dsSalaries['salary'].apply(lambda x: pd.Series(x.split('-')))

In [664]:
dsSalaries2
dsSalaries2[0] = dsSalaries2[0].str.replace(r'\D+', '').astype('int')

In [665]:
dsSalaries2[1].fillna(dsSalaries2[0], inplace=True)

In [666]:
dsSalaries2[1] = dsSalaries2[1].str.replace(r'\D+', '')
dsSalaries2[1].fillna(dsSalaries2[0], inplace=True)

In [667]:
dsSalaries2[1] = dsSalaries2[1].astype(int)
dsSalaries2.dtypes

0    int64
1    int64
dtype: object

In [668]:
#dsSalaries2['meanSalary'] = dsSalaries2[0] + dsSalaries2[1] / 2
#dsSalaries2
dsSalaries2['meanSalary'] = dsSalaries2.mean(axis=1)

In [673]:
dfSal = dsSalaries.join(dsSalaries2)
dfSal.drop(['salary', 0, 1], axis=1, inplace=True)

print dfSal.columns

Index([u'city', u'company', u'summary', u'title', u'meanSalary'], dtype='object')


In [686]:
dfSal.columns = [u'city', u'company', u'description', u'jobTitle', u'salary']
dfSal.head()

,city,company,description,jobTitle,salary
0,Washington DC,Department Of Education,Analyze data of extreme complexity largely thr...,"Statistician (Predictive Analytics), GS-1530-1...",116850.0
1,Washington DC,Department Of The Interior,Data collection tools; Identify precise operat...,"Education Research Analyst, GS-1730-13",105969.5
2,Washington DC,Legislative Branch,Or in the social sciences including demography...,Senior Statistician,101701.5
3,Washington DC,Department Of Commerce,Analyze and evaluate data. Assist in data coll...,"Recent Graduate Statistician GS-1530-7/9/11, C...",63864.0
4,Washington DC,Jobspring Partners,The data scientist will develop algorithms and...,Mid Level Data Scientist,140000.0


In [687]:
dfSal['city'] = dfSal['city'].str.replace('+', ' ')
dfSal.head()

,city,company,description,jobTitle,salary
0,Washington DC,Department Of Education,Analyze data of extreme complexity largely thr...,"Statistician (Predictive Analytics), GS-1530-1...",116850.0
1,Washington DC,Department Of The Interior,Data collection tools; Identify precise operat...,"Education Research Analyst, GS-1730-13",105969.5
2,Washington DC,Legislative Branch,Or in the social sciences including demography...,Senior Statistician,101701.5
3,Washington DC,Department Of Commerce,Analyze and evaluate data. Assist in data coll...,"Recent Graduate Statistician GS-1530-7/9/11, C...",63864.0
4,Washington DC,Jobspring Partners,The data scientist will develop algorithms and...,Mid Level Data Scientist,140000.0


In [688]:
dfSal = dfSal[dfSal['salary'] >= 20000.0]

city            object
company         object
description     object
jobTitle        object
salary         float64
dtype: object

In [708]:
bins = [0.0, 120000.0, 400000.0]
group_names = ['moderate', 'high']
category = pd.cut(dfSal['salary'], bins, labels=group_names)

KeyError: 'category'

In [709]:
dfSal.dtypes

city             object
company          object
description      object
jobTitle         object
salary          float64
salCategory    category
dtype: object

Variable | Description | Variable Type
--|--|--
city | The city in which the posted job is located | Object
company | The name of the company that posted the job | Object
description | The company's description of the job's tasks and responsibilities | Object
jobTitle | The official name of the job | Object
salary | The estimated salary for the job. These are mostly an average of a range given by Indeed.com | Float
salCategory | The category a job's salary falls. Moderate is below $120,000, while high is anything above | Category

### Write 4 functions to extract each item: location, company, job, and salary.

example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- Make sure these functions are robust and can handle cases where the data/field may not be available.
- Test the functions on the results above

Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

#### Complete the following code to collect results from multiple cities and starting points. 
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

1799
1798
1798
1798


Lastly, we need to clean up salary data. 
1. Some of the salaries are not yearly but hourly, these will be useful to us for now
2. The salaries are given as text and usually with ranges.

#### Filter out the salaries that are not yearly (filter those that refer to hour)

In [ ]:
## YOUR CODE HERE

#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [ ]:
## YOUR CODE HERE

### Save your results as a CSV

In [ ]:
#dfSalary.to_csv('../assets/04-project4-assets/dataScienceSalaries.csv')

## Predicting salaries using Logistic Regression

#### Load in the the data of scraped salaries

In [ ]:
## YOUR CODE HERE

#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

In [ ]:
## YOUR CODE HERE

#### Thought experiment: What is the baseline accuracy for this model?

In [ ]:
## YOUR CODE HERE

#### Create a Logistic Regression model to predict High/Low salary using statsmodel. Start by ONLY using the location as a feature. Display the coefficients and write a short summary of what they mean.

In [ ]:
## YOUR CODE HERE

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title 
- or whether 'Manager' is in the title. 
- Then build a new Logistic Regression model with these features. Do they add any value? 


In [ ]:
## YOUR CODE HERE

#### Rebuild this model with scikit-learn.
- You can either create the dummy features manually or use the `dmatrix` function from `patsy`
- Remember to scale the feature variables as well!


In [ ]:
## YOUR CODE HERE

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy, AUC, precision and recall of the model. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.

In [ ]:
## YOUR CODE HERE

#### Compare L1 and L2 regularization for this logistic regression model. What effect does this have on the coefficients learned?

In [ ]:
## YOUR CODE HERE

In [ ]:
## YOUR CODE HERE

#### Continue to incorporate other text features from the title or summary that you believe will predict the salary and examine their coefficients

#### Take ~100 scraped entries with salaries. Convert them to use with your model and predict the salary - which entries have the highest predicted salaries?

### BONUS 

#### Bonus: Use Count Vectorizer from scikit-learn to create features from the text summaries. 
- Examine using count or binary features in the model
- Re-evaluate the logistic regression model using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [ ]:
## YOUR CODE HERE

In [ ]:
## YOUR CODE HERE

#### Re-test L1 and L2 regularization. You can use LogisticRegressionCV to find the optimal reguarlization parameters. 
- Re-test what text features are most valuable.  
- How do L1 and L2 change the coefficients?

In [ ]:
## YOUR CODE HERE